In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os 
import sys
import time
import tensorflow as tf
from tensorflow import keras

F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
#分割数据集为：0.7训练集，0.1验证集，0.2测试集

from sklearn.model_selection import train_test_split

x_train_all,x_test,y_train_all,y_test = train_test_split(
        housing.data,
        housing.target,
        random_state = 7,
        test_size=0.2)

x_train,x_valid,y_train,y_valid = train_test_split(
        x_train_all,
        y_train_all,
        random_state=11,
        test_size=0.125)

print(x_train.shape,y_train.shape)
print(x_valid.shape,y_valid.shape)
print(x_test.shape,y_test.shape)

(14448, 8) (14448,)
(2064, 8) (2064,)
(4128, 8) (4128,)


In [4]:
#归一化操作  x = (s-u)/std   减去均值，除以方差

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


x_train_scaler = scaler.fit_transform(x_train)
x_valid_scaler = scaler.transform(x_valid)
x_test_scaler = scaler.transform(x_test)

In [10]:
#tf.keras.models.Sequential

def custmoized_mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true-y_pred))





class custmorizedDenseLayer(keras.layers.Layer):
    
    def __init__(self, units, activation=None, **kwargs):
        self.units = units
        self.activation = keras.layers.Activation(activation)
        super(custmorizedDenseLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        """构建所需要的参数"""
        # x * w + b
        # input_shape:[None,a]
        # w:[a,b]
        # output_shape:[None,b]
        self.kernel = self.add_weight(name='kernel',
                                      shape = (input_shape[1], self.units),
                                     initializer = 'uniform',
                                     trainable = True)
        self.bias = self.add_weight(name='bias',
                                   shape = (self.units,),
                                   initializer = 'zeros',
                                   trainable = True)
        super(custmorizedDenseLayer, self).build(input_shape)
        
    def call(self,x):
        """正向计算"""
        return self.activation(x @ self.kernel + self.bias)
        
        
        
        
        
        
        
        
model = keras.models.Sequential([
    
    custmorizedDenseLayer(30, activation="relu", input_shape = x_train.shape[1:]),
    custmorizedDenseLayer(1)
])


model.summary()
model.compile(loss=custmoized_mse,
             optimizer="sgd",
             metrics=["accuracy"])

W0913 21:10:34.821344 15624 ag_logging.py:145] Entity <bound method custmorizedDenseLayer.call of <__main__.custmorizedDenseLayer object at 0x000001F38C52E908>> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: (unicode error) 'utf-8' codec can't decode byte 0xd5 in position 0: invalid continuation byte (tmparexju0u.py, line 6)
W0913 21:10:34.878187 15624 ag_logging.py:145] Entity <bound method custmorizedDenseLayer.call of <__main__.custmorizedDenseLayer object at 0x000001F38BD62710>> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: (unicode error) 'utf-8' codec can't decode byte 0xd5 in position 0: invalid continuation byte (tmp40rxbd2q.py, line 6)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
custmorized_dense_layer_2 (c (None, 30)                270       
_________________________________________________________________
custmorized_dense_layer_3 (c (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(x_train_scaler, y_train, epochs = 100,
                   validation_data=(x_valid_scaler,y_valid),
                   
                   callbacks = [keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3)]
                   )

Train on 14448 samples, validate on 2064 samples
Epoch 1/100
14448/14448 [==============================] - 1s 43us/sample - loss: 0.9685 - accuracy: 0.0028 - val_loss: 0.5625 - val_accuracy: 0.0039
Epoch 2/100
14448/14448 [==============================] - 0s 26us/sample - loss: 0.5290 - accuracy: 0.0028 - val_loss: 0.4669 - val_accuracy: 0.0039
Epoch 3/100
14448/14448 [==============================] - 0s 27us/sample - loss: 0.4703 - accuracy: 0.0028 - val_loss: 0.4417 - val_accuracy: 0.0039
Epoch 4/100
14448/14448 [==============================] - 0s 26us/sample - loss: 0.4535 - accuracy: 0.0028 - val_loss: 0.4164 - val_accuracy: 0.0039
Epoch 5/100
14448/14448 [==============================] - 0s 25us/sample - loss: 0.4464 - accuracy: 0.0028 - val_loss: 0.4318 - val_accuracy: 0.0039
Epoch 6/100
14448/14448 [==============================] - 0s 25us/sample - loss: 0.4341 - accuracy: 0.0028 - val_loss: 0.4071 - val_accuracy: 0.0039
Epoch 7/100
14448/14448 [==========================